In [19]:
#数据清理
import pandas as pd

# 步骤1：读取数据
df = pd.read_csv('stay_durations_with_speed.csv')

# 步骤2和3：数据清洗和转换数据类型
# 确保'Stay Duration(seconds)'是数值类型
df['Stay Duration (seconds)'] = pd.to_numeric(df['Stay Duration (seconds)'], errors='coerce')
df['Stay Duration (hours)'] = pd.to_numeric(df['Stay Duration (hours)'], errors='coerce')
# 对每个IMSI的用户，计算停留总时长和记录数
duration_summary = df.groupby('IMSI').agg({
    'Stay Duration (hours)': 'sum',  # 停留总时长
    'IMSI': 'count'               # 记录数
})



In [2]:
print(df.columns)


Index(['IMSI', 'Location', 'Arrival Time', 'Departure Time',
       'Stay Duration (hours)', 'Stay Duration (seconds)', 'Longitude',
       'Latitude', 'Speed (km/h)'],
      dtype='object')


In [7]:
#不完整用户出行点：删除只有一条居民记录并且时长小于4小时的用户数据记录，（用户仅有1条记录但停留时长大于4小时的说明用户在家，并未出行），如果用户仅有1条记录但停留时长小于4小时的说明用户出行但出行信息缺失了，这是无用数据，删除就行了，数据量也不大。
#1）删除只有一条居民记录并且时长小于4小时的用户数据记录
imsis_to_drop = duration_summary[
    (duration_summary['Stay Duration (hours)'] < 4) & 
    (duration_summary['IMSI'] == 1)
].index

# 输出删除的IMSI个数
print(f"Number of IMSIs to be dropped: {len(imsis_to_drop)}")

Number of IMSIs to be dropped: 2


In [20]:
#2）单点停留时长较小，小于5秒
df_filtered_1 = df[df['Stay Duration (seconds)'] >= 5]

# 步骤4：计算删除的记录数
original_count = len(df)  # 原始数据集中的记录数
remaining_count = len(df_filtered)  # 筛选后的记录数
dropped_count = original_count - remaining_count  # 删除的记录数

# 输出删除的记录数
print(f"Number of records dropped: {dropped_count}")



Number of records dropped: 20441


In [15]:
#3）对于同一用户的连续轨迹，若某一个轨迹点与其前一个出现的轨迹点以及后一个出现的轨迹点所构成的夹角均小于30度，则删除该轨迹点。
import numpy as np
import pandas as pd# 读取数据
from geographiclib.geodesic import Geodesic #计算夹角的库

print(df.shape)
data_dataframe = pd.DataFrame(df)
data_numpy = np.array(data_dataframe)
print(data_numpy.shape)
data_final = data_numpy[:,6:8]#经纬度数据 矩阵，根据经纬度的所在列适当修改就行
 
for i in range(2,695220):#根据行数目修改数据
    geodict1 = Geodesic.WGS84.Inverse(float(data_final[i-1][1]), float(data_final[i-1][0]), 
                                     float(data_final[i][1]), float(data_final[i][0]))
    #点1纬度、经度，点2纬度、经度
    geodict2 = Geodesic.WGS84.Inverse(float(data_final[i][1]), float(data_final[i][0]), 
                                     float(data_final[i+1][1]), float(data_final[i+1][0]))
    az1 = geodict1['azi1']
    az2 = geodict2['azi1']
    if az1<30 and az2<30:#30度角修正
        data_final[i]=['0' '0']
 
pandas_data1=pd.DataFrame(data_final,columns=['1','2'])

# 步骤4：计算删除的记录数
original_count = len(df)  # 原始数据集中的记录数
remaining_count = len(pandas_data1)  # 筛选后的记录数
dropped_count = original_count - remaining_count  # 删除的记录数

# 输出删除的记录数
print(f"Number of records dropped: {dropped_count}")



(695226, 9)
(695226, 9)
Number of records dropped: 0


In [22]:
#异常漂移点数据修正：两点速度大于400km/h删除
# 确保'Speed(km/h)'列是数值类型
df_filtered_1['Speed (km/h)'] = pd.to_numeric(df['Speed (km/h)'], errors='coerce')

# 步骤3：筛选出速度不大于400km/h的记录
df_filtered_2 = df_filtered_1[df_filtered_1['Speed (km/h)'] <= 400]

# 步骤4：计算删除的记录数
original_count = len(df_filtered_1)  # 原始数据集中的记录数
remaining_count = len(df_filtered_2)  # 筛选后的记录数
dropped_count = original_count - remaining_count  # 删除的记录数

# 输出删除的记录数
print(f"Number of records dropped for speed greater than 400 km/h: {dropped_count}")

Number of records dropped for speed greater than 400 km/h: 5388


/var/folders/yy/xl5jyrln4lddx85rv3db9s_r0000gn/T/ipykernel_19126/2759938515.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_1['Speed (km/h)'] = pd.to_numeric(df['Speed (km/h)'], errors='coerce')


In [23]:
# 步骤5：保存结果
df_filtered_2.to_csv('processed_stay_durations_with_speed.csv', index=False)

In [24]:
# 获取数据集的基本信息
print(df_filtered_2.info())

<class 'pandas.core.frame.DataFrame'>
Index: 222706 entries, 0 to 695225
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   IMSI                     222706 non-null  int64  
 1   Location                 222706 non-null  object 
 2   Arrival Time             222706 non-null  object 
 3   Departure Time           222706 non-null  object 
 4   Stay Duration (hours)    222706 non-null  float64
 5   Stay Duration (seconds)  222706 non-null  float64
 6   Longitude                222706 non-null  float64
 7   Latitude                 222706 non-null  float64
 8   Speed (km/h)             222706 non-null  float64
dtypes: float64(5), int64(1), object(3)
memory usage: 17.0+ MB
None
